In [43]:
import tweepy
from twitter_authentication import bearer_token
import time
import pandas as pd
import json
import pickle
import folium
import json
import csv
from pathlib import Path


In [2]:
def json_load(filepath):
    with open(filepath, "r") as json_file:
        data = json.load(json_file)
    return data

def json_dump(data, filepath, pretty_format = True):
    with open(filepath, 'w') as fw:
        if pretty_format:
            json.dump(data, fw, indent=2, sort_keys=True)
        else:
            json.dump(data, fw)

def pickle_dump(obj, pickle_filepath):
    with open(pickle_filepath, "wb") as f:
        pickle.dump(obj, f, protocol=2)

def pickle_load(pickle_filepath):
    with open(pickle_filepath, "rb") as f:
        obj = pickle.load(f)
    return obj

In [3]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [13]:
next_token = None
place_dict = {}
user_dict = {}
tweet_dict = {}
alz_tweets = []



place_dict, user_dict, tweet_dict, next_token, alz_tweets = get_tweets(place_dict,user_dict,tweet_dict, next_token, alz_tweets)

In [14]:
while next_token is not None:
    place_dict, user_dict, tweet_dict, next_token = get_tweets(place_dict,user_dict,tweet_dict, next_token)

In [15]:
def get_tweets(place_dict = {},
               user_dict = {},
               tweet_dict = {},
                next_token = None,
               alz_tweets = [], 
               query = 'alzheimers disease -is:retweet place_country:US',
              user_fields = ['username', 'public_metrics', 'description', 'location'],
              place_fields = ['place_type', 'geo'],
              tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
              expansions = ['author_id', 'geo.place_id'],
              start_time = '2018-01-01T00:00:00Z',
               end_time = '2020-01-15T00:00:00Z'
              ):
    place_keys= ["full_name", 'id', 'contained_within', 'country', 'country_code', 'geo', 'name', 'place_type' ]
    user_keys= ['id', 'name', 'username', 'created_at', 
            'description', "entities", 'location', 'pinned_tweet_id', 
            'profile_image_url', 'protected', 'public_metrics', 'url', 'verified', 'withheld']
    tweet_keys = ['id', 'text', 'author_id', 'context_annotations',
             'conversation_id', 'entities', 'in_reply_to_user_id', 'lang',
             'non_public_metrics', 'organic_metrics', 'possibly_sensitive',
             'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings',
             'source', 'withheld']    


    for tweet in tweepy.Paginator(client.search_all_tweets, 
                                     query = query,
                                     user_fields = user_fields,
                                     place_fields = place_fields,
                                     tweet_fields = tweet_fields,
                                     expansions = expansions,
                                     start_time = start_time,
                                     end_time = end_time,
                                  pagination_token = next_token,
                                  max_results=500):
        time.sleep(1)
        alz_tweets.append(tweet)
    
    
    # Loop through each response object
    for response in alz_tweets:
        # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
        for place in response.includes['places']:
            place_obj = {}
            for key in place_keys:
                place_obj[key] = place[key]
            place_dict[place_obj['id']] = place_obj
        for user in response.includes['users']:
            user_obj = {}
            for key in user_keys:
                user_obj[key] = user[key]
            user_dict[user_obj['id']] = user_obj

        for tweet in response.data:
            tweet_obj = {}
            for key in tweet_keys:
                tweet_obj[key] = getattr(tweet, key)
            tweet_dict[tweet_obj['id']] = tweet_obj
#     if len(alz_tweets) == 0:
#         next_token = None
#     else:
#         if 'next_token' is in alz_tweets[0].meta:
#             next_token = alz_tweets[0].meta['next_token']
    
    return place_dict, user_dict, tweet_dict, next_token, alz_tweets

    print(len(alz_tweets))


In [16]:
alz_tweets[0].meta['next_token'], alz_tweets[1].meta['next_token']

('b26v89c19zqg8o3fnlo0z1lnmdmg3djx365x2w7h9v7ul',
 'b26v89c19zqg8o3fn0gl5k3gcjq2c5zqvgf23btbxxh8d')

In [17]:
for idx, response in enumerate(alz_tweets):
    print(idx, len(response.data))

0 496
1 489
2 345


In [18]:
print(len(alz_tweets))

3


In [19]:
alz_tweets[-1].meta

{'newest_id': '1020044637640089600',
 'oldest_id': '947865213335998464',
 'result_count': 345}

In [20]:
response = alz_tweets[0]
response.includes.keys()

dict_keys(['users', 'places'])

In [21]:
json_dump(place_dict, filepath = "place.json")

In [22]:
json_dump(user_dict, filepath = "user.json")

In [23]:
json_dump(tweet_dict, filepath = "tweet.json")

In [45]:
new = pd.DataFrame.from_dict(place_dict)

In [50]:
new_transposed = new.transpose()
new_transposed


,full_name,id,contained_within,country,country_code,geo,name,place_type
a89175c4c91f45a3,"Citrus Heights, CA",a89175c4c91f45a3,[],None,None,"{'type': 'Feature', 'bbox': [-121.331355, 38.6...",None,city
3f5897b87d2bf56c,"Delaware, USA",3f5897b87d2bf56c,[],None,None,"{'type': 'Feature', 'bbox': [-75.7887564, 38.4...",None,admin
161d2f18e3a0445a,"Raleigh, NC",161d2f18e3a0445a,[],None,None,"{'type': 'Feature', 'bbox': [-78.818343, 35.71...",None,city
00c39537733fa112,"Queens, NY",00c39537733fa112,[],None,None,"{'type': 'Feature', 'bbox': [-73.962582, 40.54...",None,city
c3f37afa9efcf94b,"Austin, TX",c3f37afa9efcf94b,[],None,None,"{'type': 'Feature', 'bbox': [-97.928935, 30.12...",None,city
...,...,...,...,...,...,...,...,...
0041984014acb103,"Madison, MS",0041984014acb103,[],None,None,"{'type': 'Feature', 'bbox': [-90.205184, 32.42...",None,city
004968f036f1ed8c,"Wauregan, CT",004968f036f1ed8c,[],None,None,"{'type': 'Feature', 'bbox': [-71.926701, 41.69...",None,city
7356b662670b2c31,"Clifton, NJ",7356b662670b2c31,[],None,None,"{'type': 'Feature', 'bbox': [-74.1977277, 40.8...",None,city
76c1bb41d3de7ff1,"Huntsville, AL",76c1bb41d3de7ff1,[],None,None,"{'type': 'Feature', 'bbox': [-86.7858813, 34.5...",None,city


In [54]:
locations = new_transposed[["full_name", "geo"]]
locations

,full_name,geo
a89175c4c91f45a3,"Citrus Heights, CA","{'type': 'Feature', 'bbox': [-121.331355, 38.6..."
3f5897b87d2bf56c,"Delaware, USA","{'type': 'Feature', 'bbox': [-75.7887564, 38.4..."
161d2f18e3a0445a,"Raleigh, NC","{'type': 'Feature', 'bbox': [-78.818343, 35.71..."
00c39537733fa112,"Queens, NY","{'type': 'Feature', 'bbox': [-73.962582, 40.54..."
c3f37afa9efcf94b,"Austin, TX","{'type': 'Feature', 'bbox': [-97.928935, 30.12..."
...,...,...
0041984014acb103,"Madison, MS","{'type': 'Feature', 'bbox': [-90.205184, 32.42..."
004968f036f1ed8c,"Wauregan, CT","{'type': 'Feature', 'bbox': [-71.926701, 41.69..."
7356b662670b2c31,"Clifton, NJ","{'type': 'Feature', 'bbox': [-74.1977277, 40.8..."
76c1bb41d3de7ff1,"Huntsville, AL","{'type': 'Feature', 'bbox': [-86.7858813, 34.5..."


In [63]:
locations.geo[0]['bbox'][1]

38.663673

In [69]:
#Long, Lat is the format for the bbox first 2 nums
long = []
lat = []
for location in locations.geo:
    long.append(location['bbox'][0])
    lat.append(location['bbox'][1])

In [75]:
full_name = []
for location in locations.full_name:
    full_name.append(location)

In [39]:
m = folium.Map(location=[5,0], tiles="OpenStreetMap", zoom_start=4)

In [40]:
m

In [12]:
#Place Attributes
place_keys= ["full_name", 'id', 'contained_within', 'country', 'country_code', 'geo', 'name', 'place_type' ]
place_dict = {}
#User Attributes
user_keys= ['id', 'name', 'username', 'created_at', 
            'description', "entities", 'location', 'pinned_tweet_id', 
            'profile_image_url', 'protected', 'public_metrics', 'url', 'verified', 'withheld']
user_dict = {}
#Tweet Attributes
tweet_keys = ['id', 'text', 'author_id', 'context_annotations',
             'conversation_id', 'entities', 'in_reply_to_user_id', 'lang',
             'non_public_metrics', 'organic_metrics', 'possibly_sensitive',
             'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings',
             'source', 'withheld']
tweet_dict = {}
full_list=[]

for response in alz_tweets: 
    for place in response.includes['places']:
        place_obj = {}
        for key in place_keys:
            place_obj[key] = place[key]
        place_dict[place_obj['id']] = place_obj
    for user in response.includes['users']:
        user_obj = {}
        for key in user_keys:
            user_obj[key] = user[key]
        user_dict[user_obj['id']] = user_obj
    for tweet in response.data:
        tweet_obj = {}
        for key in tweet_keys:
            tweet_obj[key] = getattr(tweet, key)
        tweet_dict[tweet_obj['id']] = tweet_obj
    full_list.append(place_obj)
    full_list.append(user_obj)
    full_list.append(tweet_obj)
    
        
    

In [13]:
full_list

[{'full_name': 'San Marcos, CA',
  'id': 'a2c84129f9dcf69f',
  'contained_within': [],
  'country': None,
  'country_code': None,
  'geo': {'type': 'Feature',
   'bbox': [-117.2301723, 33.0907611, -117.103461, 33.186722],
   'properties': {}},
  'name': None,
  'place_type': 'city'},
 {'id': 857761126305730560,
  'name': 'Amanda Brandeis',
  'username': 'amanda_brandeis',
  'created_at': None,
  'description': 'Visual Storyteller + Emmy Award Winning Journalist',
  'entities': None,
  'location': 'San Diego, CA',
  'pinned_tweet_id': None,
  'profile_image_url': None,
  'protected': None,
  'public_metrics': {'followers_count': 1082,
   'following_count': 530,
   'tweet_count': 1005,
   'listed_count': 21},
  'url': None,
  'verified': None,
  'withheld': None},
 {'id': 981221671792123904,
  'text': "A hiker has officially set off from San Diego on a 2,650-mile hike. \n\nHe's raising money for a disease that robs millions of their memories. More info on how to follow his journey or don